In [1]:
using IJulia
installkernel("julia_ITensors","--sysimage=~/.julia/sysimages/sys_itensors.so")

[ Info: Installing julia_ITensors kernelspec in /Users/mschuylerm/Library/Jupyter/kernels/julia_itensors-1.9


"/Users/mschuylerm/Library/Jupyter/kernels/julia_itensors-1.9"

In [2]:
using ITensors
using ITensors.HDF5
using DataFrames

# Description

In this notebook I run DMRG for multiple 1D spin systems. I look at spin chain lengths of $L = 10, 100$ for the Ising Model, the Transverse Field Ising Model, and the Heisenberg Model. I am working on the J1-J2 Model. After running DMRG, I save the optimized MPS to this local directory and then copy it over to the appropriate directory in my mps-rnn repository as an 'init.hdf5' file that gets used for hierarchical initialization of an mps-rnn.

# Spin Chain

In [3]:
function spinchain_open(L)
    
    interactions = DataFrame(["$spin"=>[] for spin in 1:L-1])
    
    for spin in 1:L-1
        neighbor = spin+1
        push!(interactions[!,"$spin"],[spin,neighbor])
    end

    return interactions
end

spinchain_open (generic function with 1 method)

In [4]:
function spinchain_peri(L)
    
    interactions = DataFrame(["$spin"=>[] for spin in 1:L])
    
    for spin in 1:L-1
        neighbor = spin+1
        push!(interactions[!,"$spin"],[spin,neighbor])
    end
    
    push!(interactions[!,"$L"],[10,1])
    
    return interactions
end

spinchain_peri (generic function with 1 method)

# Ising Model

In [7]:
function dmrg_1D_ising(L::Int, J, maxbd::Int, bc::String, save::Bool)
    
    if bc == "open"
        interactions = spinchain_open(L)
    elseif bc == "peri"
        interactions = spinchain_peri(L)
    end
    
    sites = siteinds("S=1/2",L)
    println("\nRunning DMRG for a length $L spin chain\n")
    
    # Build Hamiltonian
    os = OpSum()
    for k=1:ncol(interactions)
        i = interactions[!,k][1][1]
        j = interactions[!,k][1][2]
        os += 4*J,"Sz",i,"Sz",j
    end
    H = MPO(os,sites)

    psi0 = randomMPS(sites, 40)

    sweeps = Sweeps(10)
    setmaxdim!(sweeps, 10,20,100,100,maxbd)
    setcutoff!(sweeps, 1E-10)

    energy, psi = dmrg(H, psi0, sweeps, outputlevel=1)
    energy_per_spin = energy / L
    
    println("\nA sample from the optimized MPS looks like:\n",sample(psi))
    
    if save   
        f = h5open("../out/ising_fm_1d_L$(L)/sanity_check/init.hdf5","w") 
        write(f,"psi",psi)
        close(f)
    end
    
    for i =1:L
        println("M",i)
        println(psi[i])
    end
    
    return energy_per_spin
end

dmrg_1D_ising (generic function with 1 method)

In [8]:
dmrg_1D_ising(10,-1.0,2,"open",true);


Running DMRG for a length 10 spin chain

After sweep 1 energy=-8.999999568777453  maxlinkdim=9 maxerr=6.13E-05 time=0.012
After sweep 2 energy=-8.999999999999995  maxlinkdim=2 maxerr=4.00E-11 time=0.010
After sweep 3 energy=-8.999999999999996  maxlinkdim=2 maxerr=7.77E-16 time=0.007
After sweep 4 energy=-9.0  maxlinkdim=2 maxerr=2.22E-16 time=0.006
After sweep 5 energy=-9.0  maxlinkdim=2 maxerr=4.16E-32 time=0.007
After sweep 6 energy=-9.0  maxlinkdim=2 maxerr=6.42E-64 time=0.007
After sweep 7 energy=-9.0  maxlinkdim=2 maxerr=6.42E-64 time=0.023
After sweep 8 energy=-9.0  maxlinkdim=2 maxerr=6.42E-64 time=0.006
After sweep 9 energy=-9.0  maxlinkdim=2 maxerr=6.42E-64 time=0.006
After sweep 10 energy=-9.0  maxlinkdim=2 maxerr=6.42E-64 time=0.006

A sample from the optimized MPS looks like:
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
M1
ITensor ord=2
Dim 1: (dim=2|id=62|"Link,l=1")
Dim 2: (dim=2|id=162|"S=1/2,Site,n=1")
NDTensors.Dense{Float64, Vector{Float64}}
 2×2
 0.918381649472778      -8.0048368

# Transverse Field Ising Model

In [11]:
function dmrg_1D_tfim(L::Int, J, h, maxbd::Int, bc::String, save::Bool)
    
    if bc == "open"
        interactions = spinchain_open(L)
    elseif bc == "peri"
        interactions = spinchain_peri(L)
    end
    
    sites = siteinds("S=1/2",L)
    println("\nRunning DMRG for a length $L spin chain\n")
    
    # Build Hamiltonian
    os = OpSum()
    for k=1:ncol(interactions)
        i = interactions[!,k][1][1]
        j = interactions[!,k][1][2]
        os += 4*J,"Sz",i,"Sz",j
    end
    for k=1:L
        os += 2*h,"Sx",k
    end
    H = MPO(os,sites)

    psi0 = randomMPS(sites, 40)

    sweeps = Sweeps(10)
    setmaxdim!(sweeps, 10,20,100,100,maxbd)
    setmindim!(sweeps,maxbd)
    setcutoff!(sweeps, 1E-10)

    energy, psi = dmrg(H, psi0, sweeps, outputlevel=1)
    energy_per_spin = energy / L
    
    println("\nA sample from the optimized MPS looks like:\n",sample(psi))
    
    if save   
        f = h5open("../out/ising_fm_1d_L$(L)_h1/init.hdf5","w") 
        write(f,"psi",psi)
        close(f)
    end
    
    println("")
    @show psi
    
    return energy_per_spin
end

dmrg_1D_tfim (generic function with 1 method)

In [14]:
dmrg_1D_tfim(300,-1.,-1.,100,"open",false)


Running DMRG for a length 300 spin chain

After sweep 1 energy=-381.4910816512335  maxlinkdim=10 maxerr=5.04E-03 time=0.852
After sweep 2 energy=-381.569351611117  maxlinkdim=20 maxerr=2.19E-10 time=1.049
After sweep 3 energy=-381.57703147919364  maxlinkdim=100 maxerr=1.64E-14 time=17.716
After sweep 4 energy=-381.58181358853176  maxlinkdim=100 maxerr=1.50E-14 time=26.299
After sweep 5 energy=-381.58586758076984  maxlinkdim=100 maxerr=3.29E-14 time=26.539
After sweep 6 energy=-381.5893463058082  maxlinkdim=100 maxerr=8.95E-15 time=27.466
After sweep 7 energy=-381.5923143780902  maxlinkdim=100 maxerr=7.20E-15 time=28.280
After sweep 8 energy=-381.5948011511548  maxlinkdim=100 maxerr=4.61E-15 time=28.073
After sweep 9 energy=-381.5969086256644  maxlinkdim=100 maxerr=4.11E-15 time=25.320
After sweep 10 energy=-381.5987307127452  maxlinkdim=100 maxerr=6.75E-15 time=25.933

A sample from the optimized MPS looks like:
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 

-1.271995769042484

# Heisenberg Model

In [5]:
function dmrg_1D_heis(L::Int, J, maxbd::Int, bc::String, save::Bool)
    
    if bc == "open"
        interactions = spinchain_open(L)
    elseif bc == "peri"
        interactions = spinchain_peri(L)
    end
    
    sites = siteinds("S=1/2",L)
    println("\nRunning DMRG for a length $L spin chain\n")
    
    # Build Hamiltonian
    os = OpSum()
    for k=1:ncol(interactions)
        i = interactions[!,k][1][1]
        j = interactions[!,k][1][2]
    os += 4*J,"Sz",j,"Sz",k
    os += 4*J/2,"S+",j,"S-",k
    os += 4*J/2,"S-",j,"S+",k
    end
    H = MPO(os,sites)

    psi0 = randomMPS(sites, 40)

    sweeps = Sweeps(10)
    setmaxdim!(sweeps, 10,20,100,100,maxbd)
    setcutoff!(sweeps, 1E-10)

    energy, psi = dmrg(H, psi0, sweeps, outputlevel=1)
    energy_per_spin = energy / L
    
    println("\nA sample from the optimized MPS looks like:\n",sample(psi))
    
    if save   
        f = h5open("../out/heis_afm_1d_L$(L)/init.hdf5","w") 
        write(f,"psi",psi)
        close(f)
    end
        
    return energy_per_spin
end

dmrg_1D_heis (generic function with 1 method)

In [6]:
dmrg_1D_heis(10,1.,100,"open",false)


Running DMRG for a length 10 spin chain

After sweep 1 energy=-17.03157278466808  maxlinkdim=10 maxerr=3.45E-04 time=18.165
After sweep 2 energy=-17.032140826116663  maxlinkdim=20 maxerr=9.58E-11 time=0.020
After sweep 3 energy=-17.03214082722141  maxlinkdim=20 maxerr=9.18E-11 time=0.020
After sweep 4 energy=-17.032140827221404  maxlinkdim=20 maxerr=9.16E-11 time=0.027
After sweep 5 energy=-17.032140827221394  maxlinkdim=20 maxerr=9.16E-11 time=0.051
After sweep 6 energy=-17.032140827221415  maxlinkdim=20 maxerr=9.16E-11 time=0.040
After sweep 7 energy=-17.03214082722141  maxlinkdim=20 maxerr=9.16E-11 time=0.019
After sweep 8 energy=-17.03214082722139  maxlinkdim=20 maxerr=9.16E-11 time=0.043
After sweep 9 energy=-17.03214082722138  maxlinkdim=20 maxerr=9.16E-11 time=0.019
After sweep 10 energy=-17.032140827221383  maxlinkdim=20 maxerr=9.16E-11 time=0.020

A sample from the optimized MPS looks like:
[2, 1, 1, 2, 1, 1, 2, 2, 1, 2]


-1.7032140827221383

# J1-J2 Model?

work on this later...

In [89]:
function spinchain_J1J2(L::Int,j1::Float64,j2::Float64)
    
    interactions = DataFrame(["$spin"=>[] for spin in 1:L-1])
    
    for spin in 1:L-1
        nneighbor = spin+1
        push!(interactions[!,"$spin"],[spin,nneighbor,j1])
    end

    for spin in 1:L-2
        nnneighbor = spin+1
        push!(interactions[!,"$spin"],[spin,nnneighbor,j2])
    end

    return interactions
end

spinchain_J1J2 (generic function with 2 methods)

In [90]:
spinchain_J1J2(10,1.,2.)

AssertionError: AssertionError: Data frame is corrupt: length of column :9 (2) does not match length of column 1 (3). The column vector has likely been resized unintentionally (either directly or because it is shared with another data frame).